<a href="https://colab.research.google.com/github/Peter-Dunbar/MathAid-TinyLlama/blob/main/TinyLlama_MathAid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch transformers datasets accelerate peft huggingface_hub

In [ ]:
from datasets import load_dataset

dataset = load_dataset("HuggingFaceH4/MATH-500")
print(dataset)


DatasetDict({
    test: Dataset({
        features: ['problem', 'solution', 'answer', 'subject', 'level', 'unique_id'],
        num_rows: 500
    })
})


In [ ]:
def format_example(example):
  return {
      "prompt": f"{example['problem']}\nAnswer step by step:",
      "completion": f"{example['solution']}"
  }

dataset = dataset.map(format_example)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
def tokenize(batch):
  inputs = tokenizer(batch["prompt"], truncation=True)
  labels = tokenizer(batch["answer"], truncation=True)
  inputs["labels"] = labels["input_ids"]
  return inputs

tokenized_dataset = dataset.map(tokenize, batched=True, remove_columns=dataset["test"].features.keys())

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./tinyllama-math",
    evaluation_strategy="steps",
    save_strategy="steps",
    save_steps=100,
    logging_steps=10,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    learning_rate=2e-5,
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"].select(range(2000)),
    eval_dataset=tokenized_dataset["test"].select(range(1000)),
)

trainer.train()

KeyboardInterrupt: 

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model="./tinyllama-math", tokenizer=tokenizer)

question = "what is the derivative of x^3 + 2x?"
response = pipe(f"Question: {question}\nAnswer step by step:", max_new_tokens=200, temperature=0.2)
print(response[0]["generated_text"]